# Тема “Создание признакового пространства”

Используем предобработанные в рамках 1-ого домашнего задания датасет combine_df_prepocessed.pkl. Используем столбец 'clean_tweet'.

**Задание 1.**

Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)
С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?
Повторим шаги из заданий 1 и 2, используя библиотеку nltk.

**Задание 2.**

Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.
С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
import nltk

# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

In [3]:
import spacy 
from spacy import displacy

nlp = spacy.load('en_core_web_sm')
nlp.max_length = 1000000

**Функции, которые возвращают датафреймы с NE, найденными spacy и nltk во входном массиве текстов:**

In [4]:
def ner_by_spacy(texts):

    entities = []
    labels = []
    for text in tqdm(texts):
        doc = nlp(text)
        for ent in doc.ents:
            entities.append(ent.text)
            labels.append(ent.label_)

    df = pd.DataFrame({'Entities': entities, 'Labels': labels})

    return df

In [5]:
def ner_by_nltk(texts):
    
    entities = []
    labels = []
    
    for text in tqdm(texts):
        words = nltk.word_tokenize(text)
        pos_tags = nltk.pos_tag(words)
        chunks = nltk.ne_chunk(pos_tags, binary=False)

        for chunk in chunks:
            if hasattr(chunk, 'label'):
                entities.append(' '.join(c[0] for c in chunk))
                labels.append(chunk.label())
    
    df = pd.DataFrame({'Entities': entities, 'Labels': labels})

    return df

Статья для проверки функций:

In [6]:
test_article = "The decorated ceilings of the Natural History Museum in South Kensington, London, \
    were designed by the museum's architect Alfred Waterhouse, and were unveiled at the building's \
    opening in 1881. The ceiling of the large Central Hall (pictured) consists of 162 panels, 108 of \
    which depict plants considered significant to the history of the museum, to the British Empire or \
    to the museum's visitors. The remaining 54 are highly stylised decorative botanical paintings. The \
    ceiling of the smaller North Hall consists of 36 panels, 18 of which depict plants growing in the \
    British Isles. Both ceilings make extensive use of gilding for visual effect. Built of lath and plaster \
    to save costs, the ceilings are unusually fragile and require extensive maintenance and restoration."

In [7]:
test_sentences = nltk.sent_tokenize(test_article)

Проверка функций:

In [8]:
ner_by_spacy(test_sentences)

100%|████████████████████████████████████████████| 6/6 [00:00<00:00, 18.93it/s]


,Entities,Labels
0,the Natural History Museum,ORG
1,South Kensington,GPE
2,London,GPE
3,Alfred Waterhouse,PERSON
4,1881,DATE
5,Central Hall,FAC
6,162,CARDINAL
7,108,CARDINAL
8,the British Empire,GPE
9,54,CARDINAL


In [9]:
ner_by_nltk(test_sentences)

100%|████████████████████████████████████████████| 6/6 [00:00<00:00,  7.00it/s]


,Entities,Labels
0,Natural History Museum,ORGANIZATION
1,South Kensington,GPE
2,London,GPE
3,Alfred Waterhouse,PERSON
4,Central Hall,ORGANIZATION
5,British,GPE
6,North Hall,LOCATION
7,British Isles,GPE


### Загрузка датафрейма с твитами

In [10]:
import pickle

with open('combine_df.pickle', 'rb') as f:
    combine_df = pickle.load(f)

In [11]:
combine_df.head(n=2)

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, cause, offer, wheel..."


### NER с помощью spacy

In [12]:
df_spacy = ner_by_spacy(combine_df['tweet'])
df_spacy.head(n=10)

100%|████████████████████████████████████| 49159/49159 [24:38<00:00, 33.25it/s]


,Entities,Labels
0,wheelchair vans,ORG
1,pdx,ORG
2,bihday,PERSON
3,tomorrow,DATE
4,the next school year,DATE
5,the year,DATE
6,cleveland,GPE
7,ireland,GPE
8,orlando,GPE
9,today days,DATE


Топ-20 распознанных организаций, личностей, локаций, событий:

In [13]:
df_gb = df_spacy.groupby(['Entities', 'Labels'])[['Entities']].count()
df_gb.columns = ['counts']
df_gb.reset_index(inplace=True)
df_gb = df_gb[df_gb['Labels'].isin(['ORG', 'PERSON', 'GPE', 'FAC', 'EVENT'])]
df_gb.sort_values(by='counts', ascending=False).head(n=20)

,Entities,Labels,counts
8618,orlando,GPE,385
441,america,GPE,207
6734,london,GPE,184
12084,us,GPE,136
4687,hea,PERSON,119
1635,bong bing,ORG,107
6606,libtard sjw,GPE,100
3730,florida,GPE,95
4962,hu,PERSON,87
661,app,ORG,84


### NER с помощью nltk

In [14]:
df_nltk = ner_by_nltk(combine_df['tweet'])
df_nltk.head(n=10)

100%|████████████████████████████████████| 49159/49159 [11:32<00:00, 71.02it/s]


,Entities,Labels


К сожалению, как NE ничего не распозналось - судя по всему, из-за нижнего регистра. Тем не менее, выше показано, что функция работает.